In [1]:
# Goal : To classify the type of ovarian cancer from microscopy scans of biopsy samples.

# Search SoTA model on Kaggle Approach, Papers with code - Medical Data, Graph Transformer, GNN, Global and Local Analysis

# You should manage, including differences in image dimensions, quality, slide staining techniques, and more

# There are 2 types of images - TMA and WSI (25 images are TMA and 513(rest) are WSI)


In [2]:
import sys
sys.prefix

'/projectnb/cs640grp/students/avarshn/.conda/envs/ocean'

# Check if GPU is available

In [3]:
import torch

In [4]:
torch.cuda.is_available()

False

In [5]:
!nvidia smi


/bin/bash: nvidia: command not found


In [6]:
!lspci | grep -i nvidia

# Import Libraries

In [7]:
import os
from time import time
from collections import defaultdict
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

from PIL import Image
import cv2

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.optim import AdamW

from torchsummary import summary

import torchvision
from torchvision import transforms, utils
from torchvision.transforms import v2

from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torchvision.models import efficientnet_b3, EfficientNet_B3_Weights
from torchvision.models import resnet18, ResNet18_Weights

# Configurations

In [8]:
# https://github.com/pytorch/vision/issues/7744

from torchvision.models._api import WeightsEnum
from torch.hub import load_state_dict_from_url

def get_state_dict(self, *args, **kwargs):
    kwargs.pop("check_hash")
    return load_state_dict_from_url(self.url, *args, **kwargs)
WeightsEnum.get_state_dict = get_state_dict

In [9]:
CONFIG = {
    "model_name" : "efficientnet_b0"
}

In [10]:
if CONFIG["model_name"] == "efficientnet_b3":
    model_weights = EfficientNet_B3_Weights.DEFAULT
    base_model = efficientnet_b3(weights = model_weights)

    CONFIG.update({
            "seed": 42,
            
            "num_classes": 5,
            # "valid_batch_size": 32,
            "resize_size": (320, 320),
            "embedding_save_folder" : "./models/efficientnet_b3/embeddings/",
            "batch_size" : 16,
            "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    })
    
    try:
        os.makedirs(CONFIG["embedding_save_folder"])
    except:
        print("Unable to create a directory")

In [11]:
if CONFIG["model_name"] == "efficientnet_b0":
    model_weights = EfficientNet_B0_Weights.DEFAULT
    base_model = efficientnet_b0(weights = model_weights)

    CONFIG.update({
            "seed": 42,
            
            "num_classes": 5,
            # "valid_batch_size": 32,
            "resize_size": (256, 256),
            "embedding_save_folder" : "./models/efficientnet_b0/embeddings/",
            "batch_size" : 16,
            "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    })

In [12]:
if CONFIG["model_name"] == "resnet18":
    model_weights = ResNet18_Weights.DEFAULT
    base_model = resnet18(weights = model_weights)

    CONFIG.update({
            "seed": 42,
            
            "num_classes": 5,
            # "valid_batch_size": 32,
            "resize_size": (256, 256),
            "embedding_save_folder" : "./models/resnet18/embeddings/",
            "batch_size" : 16,
            "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    })

In [13]:
CONFIG

{'model_name': 'efficientnet_b0',
 'seed': 42,
 'num_classes': 5,
 'resize_size': (256, 256),
 'embedding_save_folder': './models/efficientnet_b0/embeddings/',
 'batch_size': 16,
 'device': device(type='cpu')}

In [14]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

# Loading Data

In [15]:
SCC_folder = "/projectnb/cs640grp/"
data_folder = "materials/UBC-OCEAN_CS640/"

In [16]:
data_path = SCC_folder + data_folder

In [17]:
os.listdir(data_path)

['test_script.py',
 'test_images_compressed_80',
 'train.csv',
 'test.csv',
 'train_images_compressed_80',
 'all_labels.npy']

In [18]:
labels = np.load(data_path +  'all_labels.npy')
labels

array(['HGSC', 'EC', 'MC', 'CC', 'LGSC'], dtype='<U4')

In [19]:
train_df = pd.read_csv(data_path +  'train.csv')
train_df["image_path"] = train_df["image_id"].apply(lambda x : str(x) + ".jpg")
train_df.head()

,image_id,label,image_path
0,57598,CC,57598.jpg
1,30868,MC,30868.jpg
2,42549,CC,42549.jpg
3,64824,CC,64824.jpg
4,15293,HGSC,15293.jpg


In [20]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_id    430 non-null    int64 
 1   label       430 non-null    object
 2   image_path  430 non-null    object
dtypes: int64(1), object(2)
memory usage: 10.2+ KB


In [21]:
train_df["image_id"].nunique()

430

## Training Data Distribution

In [22]:
train_df["label"].value_counts()

label
HGSC    177
EC       99
CC       79
LGSC     38
MC       37
Name: count, dtype: int64

In [23]:
100*train_df["label"].value_counts(normalize = True)

label
HGSC    41.162791
EC      23.023256
CC      18.372093
LGSC     8.837209
MC       8.604651
Name: proportion, dtype: float64

In [24]:
test_df = pd.read_csv(data_path +  'test.csv')
test_df["image_path"] = test_df["image_id"].apply(lambda x : str(x) + ".jpg")
test_df.head()

,image_id,label,image_path
0,0,NaN,0.jpg
1,1,NaN,1.jpg
2,2,NaN,2.jpg
3,3,NaN,3.jpg
4,4,NaN,4.jpg


In [25]:
# There is class imbalance

# high-grade serous carcinoma, clear-cell ovarian carcinoma, endometrioid, low-grade serous, and mucinous carcinoma

# Can use data augmentation to resolve this as slide can be in any fashion so can rotate the slides

# Custom Preprocessing

In [26]:
Image.MAX_IMAGE_PIXELS = None

In [27]:
# Default - Model Transformation in PyTorch documentation - it will be used for resizing to the correct size and normalization process
default_transform = model_weights.transforms()

color_jitter = transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2)

# Augmentation preprocessing - Augmentation on the fly
augmentation_transform = transforms.Compose([
        # Resize
        transforms.Resize(CONFIG["resize_size"]),

        # Rotation and Flip
        transforms.RandomRotation(degrees=[0, 90]),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
    
        # To make it robust to different slightly changes in slide staining techniques from each lab
        transforms.RandomApply([color_jitter], p=0.8)
    
        # Add other augmentation transforms as needed
])

# Combine both transforms - Training
def combined_train_transform(img):
    # print("1")
    img = augmentation_transform(img)
    # print("2")
    img = default_transform(img)
    # print("End")
    return img

# Testing - No need to augment
def test_transform(img):
    # img = augmentation_transform(img)
    img = default_transform(img)
    return img

In [28]:
# preprocess = transforms.Compose([
#         # transforms.RandomSizedCrop(224),
#         # transforms.RandomHorizontalFlip(),
#         transforms.Resize((4000, 4000)),
#         transforms.ToTensor(),   # Converts a PIL Image or a NumPy array with values in the range [0, 255] to a PyTorch tensor
#                                  # with values in the range [0.0, 1.0].
#         # transforms.Normalize(mean=[0.485, 0.456, 0.406],
#         #                      std=[0.229, 0.224, 0.225])
# ])

In [29]:
# import albumentations as A
# def apply_augmentation(img):
#     # Augmentation using albumentations library
#     transform = A.Compose([
#         A.RandomRotate90(),
#         A.Flip(),
#         A.Transpose(),
#         A.OneOf([
#             A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2),
#             A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20),
#         ], p=0.5),
#         A.OneOf([
#             A.Blur(blur_limit=3),
#             A.MotionBlur(blur_limit=3),
#             A.GaussNoise(var_limit=(5.0, 30.0)),
#         ], p=0.5),
#     ])
#     augmented = transform(image=img)
#     return augmented['image']

In [30]:
# def apply_augmentation(img):
#     # transforms = v2.Compose([
#     #     v2.RandomRotate(90),
#     #     v2.RandomHorizontalFlip(p=0.5),
#     #     v2.ColorJitter(brightness=(0.01,0.4),contrast=(0.01,0.5),saturation=0.4)
#     #     v2.RandomApply([
#     #         v2.GaussianBlur(
#     #     ]p=0.5),
#     #     v2.ToDtype(torch.float32, scale=True),
#     #     v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     # ])
#     transforms = v2.AutoAugment(policy=AutoAugmentPolicy.IMAGENET)
#     img = transforms(img)

# Custom Dataset

In [31]:
class ocean_dataset(Dataset):
    # def transform(img):
    #     return apply_augmentation(img)
    
    def __init__(self, dataframe, image_dir, transform = None):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.ocean_df = dataframe
        self.ocean_df["image_path"] = self.ocean_df["image_id"].apply(lambda x : str(x) + ".jpg")
        
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.ocean_df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = os.path.join(self.image_dir, self.ocean_df["image_path"][idx])
        image = Image.open(img_path)

        # # Convert Pillow image to NumPy array
        # img_array = np.array(image)
        
        # # Display the range of pixel values
        # print(f"Minimum pixel value: {np.min(img_array)}")
        # print(f"Maximum pixel value: {np.max(img_array)}")


        if self.transform:
            image = self.transform(image)

        label_mapper = {'HGSC' : 0, 'EC' : 1, 'MC' : 2, 'CC' : 3, 'LGSC' : 4}
        label = label_mapper[self.ocean_df.iloc[idx, 1]]

        
        # sample = {'image': image, 'label': label}

        return image, label
        

In [32]:
df = pd.read_csv(data_path +  'train.csv')

# For testing the pipeline
# df = df[0:30]

labels_for_stratifying = df["label"].values

train_df, test_df = train_test_split(df, test_size = 0.3, random_state = CONFIG['seed'], stratify = labels_for_stratifying)

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [33]:
train_df["label"].value_counts()/301

label
HGSC    0.411960
EC      0.229236
CC      0.182724
LGSC    0.089701
MC      0.086379
Name: count, dtype: float64

In [34]:
test_df["label"].value_counts()/129

label
HGSC    0.410853
EC      0.232558
CC      0.186047
MC      0.085271
LGSC    0.085271
Name: count, dtype: float64

In [35]:
train_dataset = ocean_dataset(train_df, data_path + 'train_images_compressed_80/', combined_train_transform)

test_dataset = ocean_dataset(test_df, data_path + 'train_images_compressed_80/', test_transform)

In [36]:
a,b = train_dataset.__getitem__(torch.tensor(8))

In [37]:
b

0

# Class Weights - From Training Data since the dataset is imbalanced

In [38]:
train_df["label"]

0        EC
1      HGSC
2        EC
3      LGSC
4      HGSC
       ... 
296      MC
297    HGSC
298      EC
299    HGSC
300    HGSC
Name: label, Length: 301, dtype: object

In [39]:
label_mapper = {'HGSC' : 0, 'EC' : 1, 'MC' : 2, 'CC' : 3, 'LGSC' : 4}
labels = train_df["label"].apply(lambda x: label_mapper[x])
labels_arr = labels.values

In [40]:
train_df["label"].value_counts()

label
HGSC    124
EC       69
CC       55
LGSC     27
MC       26
Name: count, dtype: int64

In [41]:
# Formula -
# class_weight_of_that_class = n_samples / (n_classes * n_samples_of_that_class)

In [42]:
# NumPy array of class labels - labels_arr

class_weights = compute_class_weight('balanced',classes = np.unique(labels_arr), y = labels_arr)

In [43]:
class_weights

array([0.48548387, 0.87246377, 2.31538462, 1.09454545, 2.22962963])

In [44]:
class_weights_tensor = torch.tensor(class_weights)
class_weights_tensor

tensor([0.4855, 0.8725, 2.3154, 1.0945, 2.2296], dtype=torch.float64)

# Calculate Sample weights for Training data
- sample_weights is a list of weightage given to the idx in the training data while sampling
- So, we will assign higher sample weights to labels which are relatively less in our training data, this will make our data balanced.

In [45]:
# # This takes a lot of time - since we are applying certain transformation

# sample_weights = []
# for idx, (image, label) in enumerate(train_dataset):
#     sample_weights.append(class_weights[label])


# Instead since we are using train_df to create train_dataset, we can use train_df labels directly as we need labels only
sample_weights = []
for label_class in train_df["label"]:
    label = label_mapper[label_class]
    sample_weights.append(class_weights[label])

In [46]:
print(len(sample_weights))
print(sample_weights[0:10])

301
[0.8724637681159421, 0.4854838709677419, 0.8724637681159421, 2.22962962962963, 0.4854838709677419, 0.4854838709677419, 0.8724637681159421, 2.3153846153846156, 0.4854838709677419, 1.0945454545454545]


In [47]:
print(label_mapper, "\n", class_weights)

{'HGSC': 0, 'EC': 1, 'MC': 2, 'CC': 3, 'LGSC': 4} 
 [0.48548387 0.87246377 2.31538462 1.09454545 2.22962963]


In [48]:
train_df[0:10]

,image_id,label,image_path
0,59760,EC,59760.jpg
1,17174,HGSC,17174.jpg
2,51128,EC,51128.jpg
3,31793,LGSC,31793.jpg
4,64188,HGSC,64188.jpg
5,30508,HGSC,30508.jpg
6,21910,EC,21910.jpg
7,48550,MC,48550.jpg
8,52275,HGSC,52275.jpg
9,16876,CC,16876.jpg


# Generate Embeddings

In [49]:
for i, layer in enumerate(base_model.children()):
    print(i , layer)
    print("*-"*20)

0 Sequential(
  (0): Conv2dNormActivation(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): SiLU(inplace=True)
  )
  (1): Sequential(
    (0): MBConv(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): SiLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (activation): SiLU(inplace=True)
          (scale_activation): Sigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1)

In [50]:
if CONFIG["model_name"] == "efficientnet_b3":
    base_layers = nn.Sequential()
    for i, layer in enumerate(base_model.children()):
        if i <2:
            base_layers.add_module(str(i), layer)

if CONFIG["model_name"] == "efficientnet_b0":
    base_layers = nn.Sequential()
    for i, layer in enumerate(base_model.children()):
        if i <2:
            base_layers.add_module(str(i), layer)

if CONFIG["model_name"] == "resnet18":
    base_layers = nn.Sequential()
    for i, layer in enumerate(base_model.children()):
        if i <9:
            base_layers.add_module(str(i), layer)

In [51]:
class Embedding_Model(nn.Module):

    def __init__(self):
        super().__init__()
        
        self.base_model_headless = base_layers
        self.base_model_headless.requires_grad = False

    def forward(self, x):
        x = self.base_model_headless(x)
        # print("Output shape after Passing through Pretrained Model")
        # print(x.shape)

        # Flatten
        x = x.view((x.shape[0], -1))

        return x      

In [52]:
# Initialize Model
model = Embedding_Model().to(CONFIG["device"])

In [53]:
summary(model, input_size = (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              SiLU-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
              SiLU-6         [-1, 32, 112, 112]               0
 AdaptiveAvgPool2d-7             [-1, 32, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]             264
              SiLU-9              [-1, 8, 1, 1]               0
           Conv2d-10             [-1, 32, 1, 1]             288
          Sigmoid-11             [-1, 32, 1, 1]               0
SqueezeExcitation-12         [-1, 32, 112, 112]               0
           Conv2d-13         [-1, 16, 112, 112]             512
      BatchNorm2d-14         [-1, 16, 1

In [54]:
len(train_df)

301

In [59]:
# Create a weighted sampler - To balance the dataset
data_scaling_factor = 1
# replacement = True because we want to oversample minority class
sampler = WeightedRandomSampler(sample_weights, num_samples = data_scaling_factor * len(train_df), replacement=True)
# sampler = WeightedRandomSampler(sample_weights, num_samples = 8, replacement=True)  # Testing purposes

# ValueError: sampler option is mutually exclusive with shuffle
# When using data augmentation, the data is preprocessed before being loaded onto the GPU, the augmentation is typically applied on the CPU. 
# However, you can still achieve some level of parallelism by using multi-worker data loading with num_workers in the DataLoader. 
# Additionally, setting pin_memory=True can speed up the transfer of data from the CPU to the GPU.
trainloader = DataLoader(train_dataset, batch_size = CONFIG['batch_size'], drop_last = False, sampler = sampler, num_workers = 8, pin_memory =True)

testloader = DataLoader(test_dataset, batch_size = CONFIG['batch_size'], shuffle = False, drop_last = False, num_workers = 8, pin_memory =True)

In [60]:
# %%time
# # Testing
# for i, (images, labels) in enumerate(trainloader, 1):
#     print(f"Batch {i}")

In [61]:
# Use `torch.no_grad()` here to disable gradient calculation. 
# It will reduce memory consumption as we don't need to compute gradients in inference.

@torch.no_grad()
def generate_embeddings(model, dataloader, device, save_folder, save_as, aug=0):
    
    model.eval()

    embeddings_list = []
    labels_list = []
    
    # Loop through each batch on test set
    for i, (images, labels) in enumerate(dataloader, 1):
        print(f"Batch {i}")
        
        images = images.to(device)
        labels = labels.to(device)
        
        embeddings = model(images)

        embeddings_list.append(embeddings.numpy())
        labels_list.append(labels.numpy())


    all_embeddings = np.concatenate(embeddings_list, axis = 0)
    all_labels = np.concatenate(labels_list, axis = 0)
    

    # Save embeddings and labels to a NumPy file
    if(aug != 0):
        np.savez(f"{save_folder}{save_as}_embeddings_and_labels_{aug}.npz", embeddings = all_embeddings, labels = all_labels)
    else:        
        np.savez(f"{save_folder}{save_as}_embeddings_and_labels.npz", embeddings = all_embeddings, labels = all_labels)

    return "Embeddings saved"


In [69]:
# Embeddings
generate_embeddings(model, testloader, device = CONFIG["device"], save_folder = CONFIG["embedding_save_folder"], save_as = f"{CONFIG['model_name']}_test", aug = 0)

Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Batch 8
Batch 9


'Embeddings saved'

In [62]:
# Embeddings
from tqdm import tqdm

for i in tqdm(range(23,51)):
    generate_embeddings(model, trainloader, device = CONFIG["device"], save_folder = CONFIG["embedding_save_folder"], save_as = f"{CONFIG['model_name']}_train",aug = i)

  0%|          | 0/28 [00:00<?, ?it/s]

Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Batch 8
Batch 9
Batch 10
Batch 11
Batch 12
Batch 13
Batch 14
Batch 15
Batch 16
Batch 17
Batch 18
Batch 19


  4%|▎         | 1/28 [01:52<50:41, 112.65s/it]

Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Batch 8
Batch 9
Batch 10
Batch 11
Batch 12
Batch 13
Batch 14
Batch 15
Batch 16
Batch 17
Batch 18
Batch 19


  7%|▋         | 2/28 [03:48<49:35, 114.43s/it]

...

 93%|█████████▎| 26/28 [51:01<03:57, 118.76s/it]

Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Batch 8
Batch 9
Batch 10
Batch 11
Batch 12
Batch 13
Batch 14
Batch 15
Batch 16
Batch 17
Batch 18
Batch 19


 96%|█████████▋| 27/28 [52:58<01:58, 118.00s/it]

Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Batch 8
Batch 9
Batch 10
Batch 11
Batch 12
Batch 13
Batch 14
Batch 15
Batch 16
Batch 17
Batch 18
Batch 19


100%|██████████| 28/28 [54:54<00:00, 117.66s/it]
